<a href="https://colab.research.google.com/github/ManassiLab/SR_Consistency/blob/main/Copia_di_Face_Id_Analysis_Split_half_analysis_Within_consistency_3BINS_newVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import Packages**

In [2]:
import sys
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import numpy
import math
import statistics
import random
import scipy.stats as stats
#from lmfit import Model
from numpy import exp, loadtxt, sqrt, std, mean
from scipy import stats
from scipy.stats import sem
from scipy.stats import pearsonr
#from lmfit.models import GaussianModel
from scipy.stats import linregress
numpy.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(threshold=np.inf)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', None)
%matplotlib inline
#%pip install lmfit
#sys.path.insert(-1, "c:\python\lib\site-packages\lmfit")

print("import packages DONE")

import packages DONE


**Parameters**

In [3]:
ActualStim = 'stimulusID'
#StartResp = 'StartBar'
Response = 'morphID'
trialsLoop = 'trialNumber' #trialnum per block trialsLoop.thisRepN
blocksLoop = 'blockNumber' #blocknum blocksLoop.thisRepN
#GaborTime = 'GaborTime'
#MaskTime = 'MaskTime'
#ISItime = 'ISI' #blocknum blocksLoop.thisRepN
#TimeResp = 'RespTime'
#ITI = 'ITI'
Location = 'stimLocationDeg'
#Gender = '*Geschlecht (M/W/anders)'
#age = '*Geburtsjahr '
#PersonalCode = 'Bitte hier eingeben: (1) die ersten 2 Buchstaben des Vornamens Ihrer Mutter, (2) der Tag (des Monats) an dem Sie geboren wurden, (3) die letzten 2 Buchstaben Ihres Vornamens, (4) der Tag (des Monats) an dem Ihre Mutter geboren wurde (Bsp: LY16KE26)'
blockType_ = 'blockType'
error_corr = 'errorCorrected'
SDcutoffYN = 0
FilterYN = 0
halfway = 74 # circular space correction (74 morphs, 90 orientation)
CurrentPath = os. getcwd()

print("Parameters DONE")
print("Current Path is        " + CurrentPath)
pathSTART = CurrentPath + "/SR_Consistency-main/Data/"
print("Looking into           " + pathSTART)


Parameters DONE
Current Path is        /content
Looking into           /content/SR_Consistency-main/Data/


**Get all files**

In [4]:

def GetFromSingleCsv(path):

### LOOK FOR FILES IN FOLDER
    files = os.listdir(path) # list of ALL files
    print("Files found in folder:", files)
    files_csv = list(filter(lambda x: x[-4:] == '.csv' , files)) # list of all CSV files

    FileNameList = []
    for file in files_csv:
        FileName = file[-7:-4]
        FileNameList.append(FileName) #to give random name to file saved NOT USED

    ##### if result exists, don't run function again
    ResultPath = path + 'results/'

    ##### show CSV files to analyze
    print('CSV FILES FOUND AND READ:',files_csv)

### DATA ASSEMBLING
    # define a void list to store data
    data_list = pd.DataFrame() #dataframe

    #read useful columns, save in all_data
    for file in  files_csv:
        tmp = (pd.read_csv(path + file)[[ActualStim, Response, error_corr, trialsLoop, blocksLoop,Location,blockType_]]) #add variables you want here!
        all_data = pd.concat([data_list, tmp],ignore_index=True) #if all together: ignore_index=True

    # computes trial number per block, block number and total trial number
    file_csv = len(files_csv) #only for online studies, total of files analyzed
    trials = (np.nanmax(all_data[trialsLoop])) #+1 because it starts at 0
    blocks = (np.nanmax(all_data[blocksLoop])) #+1 because it starts at 0

    PerFile = (trials*blocks) # Trials in each csv file
    BlocksTotal = blocks*file_csv # Blocks in all csv file
    TotalTrial = trials * blocks * file_csv # SuperTotal Trials

    print(file_csv,'file_csv(s):\n', int(trials),'trials/blocks.', int(blocks),'blocks - each file_csv.\n', int(PerFile) ,'trials - each file_csv.\n', int(BlocksTotal),'total blocks TOTAL.', int(TotalTrial),'total trials TOTAL.')

    # delete invalid rows and useless columns
    all_data.dropna(axis = 0, how = 'any', inplace = True) # exclude NaN (often between blocks)
    all_data = all_data[all_data['blockType']=='Experiment']
    #MM qui potresti fare il concatenate in 3D, cosi hai una 2d matrix per ogni subj
    return all_data


**Get Data in**

In [5]:

def DataIn(all_data, path):

    xdata = all_data['stimulusID']
    CurrStim = np.array(xdata)
    Respdata = all_data['morphID']
    ydata = all_data[error_corr] #np.array(Respdata)-np.array(xdata) # ERROR
    
#FILTER FOR OUTLIERS IN ERROR (ydata)
    if SDcutoffYN:
        ErrorCutoff = 2 * std(ydata) # otherwise ErrorCutoff at the beginning
    if FilterYN == 1:
        xdata = xdata[abs(ydata) < ErrorCutoff]
        CurrStim = CurrStim[abs(ydata) < ErrorCutoff]
        Blocks_ = Blocks_[abs(ydata) < ErrorCutoff]
        ydata = ydata[abs(ydata) < ErrorCutoff]
        std_ydata=np.std(ydata)
        std_ydata=round(std_ydata,2)
        print("Resp Err NO FILTER =",std_ydataNOFILTER,"Response Error After Outlier Removal =",std_ydata,", cutoff threshold:", int(ErrorCutoff), ",", "trials removed", int(ydataAfterConditions - len(ydata)))
    else:
        std_ydata=np.std(ydata)
        std_ydata=round(std_ydata,2)
        print("Response Error No Filter =",std_ydata)
    
    #compensates for biases
    #ydata = ydata-np.mean(ydata)---> DA FARE??  
    
#DEFINING GRAPH VARIABLES
    ydata_neg = ydata #error, with negative values   #rel_error e #abs_error
    ydata = abs(ydata) #absolute error
    df = pd.DataFrame({"ActualZ": xdata, "Error": ydata, "Error_neg": ydata_neg})

#GROUPS INTO BINS    
    slices2 = np.arange(1,147,3)
    slices2 = slices2.tolist()
    
    TotBins= []
    for x in slices2:
        bins_= df[(df["ActualZ"]>=x) & (df["ActualZ"]<x+3)]
        bins_2 = list(bins_["Error_neg"])
        TotBins.append(bins_2)                  
    TotBins_allsubjects.append(TotBins) 
    
    return TotBins_allsubjects


**Main Script**

In [6]:
#############################################Runs Function#############################################

All_ErrorOut = []
TotAge = []
TotAmplitude = []
TotAmplitude2 = []
Totmean = []
Totmean_abs = []
SEM = []
Totmean1 = []
TotBins_allsubjects = []


Subj = sorted(os.listdir(pathSTART))
print("Folders found: ", Subj)
TotalSubj = np.arange(0, len(Subj))

for i in TotalSubj:
    all_data = []
    TotBins_allsubjects = []
    path = pathSTART + Subj[i] + "/"
    print("SUBJECT FROM:", path)
    all_data = GetFromSingleCsv(path) #goes into each folder, reads csv and gets all_data
    TotBins_allsubjects = DataIn(all_data,path)
    print()
    print()
    print()


FileNotFoundError: ignored

**Within-Consistency Analysis**

**Plotting Data Within-Subject Consistency**